In [ ]:
# Set up imports
import xarray as xr
from pystac_client import Client
import stackstac


# Exploring STAC catalogs using Python

STAC is a specification for describing spatiotemporal data assets, intended to facilite search and discovery of relevant data, especially based on spatial and/or temporal queries of interest.

## CEDA STAC API

STAC API provided by the Centre for Environmental Data Analysis (UK)

STAC Browser: https://stac.ceda.ac.uk/?.language=en

First we open a connection to the root-level catalog...

In [ ]:
uri = 'https://api.stac.ceda.ac.uk/'
catalog = Client.open(uri)
catalog

Now grab all the collections in the catalog, and print out a list of the IDs and names.

In [ ]:
collections = catalog.get_all_collections()
for collection in collections:
    print(f'[{collection.id}]: {collection.title}')

Let's look closer at the monthly Arctic sea ice thickness collection.

In [ ]:
ice_collection_id = 'eocis-arctic-sea-ice-thickness-monthly'
ice_collection = catalog.get_collection(ice_collection_id)
ice_collection

This particular catalog supports `search` requests, so let's do a first search that looks for all items in the ice collection.

In [ ]:
ice_item_collection = catalog.search(
        collections=[ice_collection.id]
    ).item_collection()

print(f'Returned {len(ice_item_collection)} items')
print('First 5 items:')
ice_item_collection.items[:5]

Now let's use the bread-and-butter of STAC. We'll repeat the search, now applying a combination of spatial and temporal constraints to further limit the results.

In [ ]:
ice_item_subset = catalog.search(
    collections=[ice_collection.id],
    bbox=[-157.223175, 71.134346, -155.816925, 71.388457],
    datetime='2020'
).item_collection()

print(f'Retrieved {len(ice_item_subset)} items for collection {ice_collection.id}')
ice_item_subset

Note that we can retrieve an item by ID from the collection, where the collection has been retrieve by ID from the catalog.

In [ ]:
ice_item = catalog \
    .get_collection(ice_collection.id) \
    .get_item('ea60feff-3a21-49d1-b79f-9179c052cfc0')
ice_item

Click into the `assets` in the entry above. It's a kerchunk "dataset"! In a nutshell, this is a Zarr store in which the data references point "into" linked netCDF files, with byte ranges corresponding to the segments of the file holding the relevant "chunks" of interest. Everything connects :)

In [ ]:
fs = fsspec.filesystem('http')

kerchunk_href = ice_item.assets['reference_file'].href
with fs.open(kerchunk_href) as f:
    print(json.dumps(json.load(f), indent=2))

# Discover Sentinel COGs and create a stack

Hat tip: This tutorial is based on https://www.geodose.com/2024/02/pystac-decoded-step-by-step-tutorial.html

See online browser: https://stacindex.org/catalogs/earth-search#/

First let's connect to the top-level catalog

In [ ]:
uri = 'https://earth-search.aws.element84.com/v1'
catalog = Client.open(uri)
catalog

Now we'll see what collections are stored at that Catalog level

In [ ]:
collection_list = list(catalog.get_collections())
print(f"Catalog contains {len(collection_list)} collections")
for collection in collection_list:
    print(f'- [{collection.id}] {collection.title}')

Let's dive into the `sentinel-2-l2a` collection.

In the returned widget, look at some of the details. Especially take note of:
- **extent**: Note the information about the _spatial_ and _temporal_ coverage of this collection. This is central to STAC!
- **item_asset**
- ... and more ...

In [ ]:
sentinel_collection_id = 'sentinel-2-l2a'
catalog.get_collection(sentinel_collection_id)

What does this collection contain? For starts ... how big is the collection, in terms of the number of items?

Here's a trick to get a total item count: We do an search on the catalog, specifically referencing the collection of interest, but otherwise not providing any query constraints. Importantly, we'll tell this method not to return any items. Then we'll invoke the `matched()` method on this returned object, reporting the total count of "matched" items -- which in this case is the total count of items in the collection

In [ ]:
n = catalog.search(collections=[sentinel_collection_id], max_items=0).matched()
print(f'Found {n:,} items in the {sentinel_collection_id} collection')

In [ ]:
# For a smaller collection, we could bring back all of the item information 
# in a list. But for 40M, we don't want to do this :)
# [item.id for item in catalog.get_collection('sentinel-2-l2a').get_items()]

Now let's get down to the **item** level. Quoting the STAC spec (with emphasis added):
>An Item is a GeoJSON Feature augmented with foreign members relevant to a STAC object. These include fields that identify the time range and assets of the Item. **An Item is the core object in a STAC Catalog**, containing the core metadata that enables any client to search or crawl online catalogs of spatial 'assets' (e.g., satellite imagery, derived data, DEMs).


To retrieve some items, we'll go back to the `catalog.search()` method, but this time providing some significant restrictions on what we want. In particular, we'll limit by **geographic extent** using a (hand-created) bounding box corresponding to a small area near Utqiagvik, and by temporal extent using a date range spanning a few months.

In [ ]:
# https://linestrings.com/bbox/#-156.8,71.25,-156.7,71.3
east, west, north, south = -156.7, -156.8, 71.3, 71.25
bbox_utqiagvik = [west, south, east, north]

item_collection = catalog.search(
    #intersects=dict(type="Point", coordinates=[lon, lat]),
    collections=[sentinel_collection_id],
    bbox = bbox_utqiagvik,
    datetime="2020-03-01/2020-06-01"
).item_collection()

print(f'Returned {len(item_collection)} items')


In [ ]:
for item in item_collection.items[:10]:
    print(item.id,":", item.properties['datetime'])

Let's have a look at the first item. Now the specific geographic extent of the associated item assets are recorded in the **geometry** and **bbox** entries, and the temporal information is stored in **properties -> datetime**.

In [ ]:
item_collection.items[0]

Remember, STAC items are still _catalog entries_, not the underlying data or other artifact! STAC doesn't actually store data, but like any useful catalog, it tells you were to find. In STAC parlance, the "things" it refers to are called **assets**, which "belong" to an item. An item can refer to many assets. You can see them under the **assets** intry in the catalog visualizer above, or access them in a dictionary-like way using the `assets` accessor on the item.

In [ ]:
item_collection.items[0].assets['nir']

From the information above, we can see that the _nir_ reflectance data artifact corresponding to this asset is stored as a cloud-optimized GeoTIFF in AWS S3 storage. We can take the individual COG URL and throw it in [this cool interactive COG web mapper site](https://cholmes.github.io/cog-map/#/url/https%3A%2F%2Fsentinel-cogs.s3.us-west-2.amazonaws.com%2Fsentinel-s2-l2a-cogs%2F4%2FW%2FEE%2F2020%2F6%2FS2B_4WEE_20200601_1_L2A%2FB08.tif/center/-155.658,70.152/zoom/6.2), and could of course load this into Python using `rasterio` or `rioxarray`, depending on our preference.

Here's an example using xarrarry

In [ ]:
import rioxarray

href = "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/4/W/EE/2020/6/S2B_4WEE_20200601_1_L2A/B08.tif"
rx_nir = rioxarray.open_rasterio(href, chunks={})
# could also use xr.open_dataarray(href, engine='rasterio', chunks='auto')

rx_nir


In [ ]:
rx_nir.squeeze()[-1500:, -1500:].plot.imshow();

But what if we want to assemble a bunch of COGs for different dates? We could do it manually. Or ... `stackstac` to the rescue!

From [their documentation](https://stackstac.readthedocs.io/en/latest/api/main/stackstac.stack.html), here's what `stackstac` does for you:
> - Figure out the geospatial parameters from the STAC metadata (if possible): a coordinate reference system, resolution, and bounding box.
> - Transfer the STAC metadata into xarray coordinates for easy indexing, filtering, and provenance of metadata.
> - Efficiently generate a Dask graph for loading the data in parallel.
> - Mediate between Dask’s parallelism and GDAL’s aversion to it, allowing for fast, multi-threaded reads when possible, and at least preventing segfaults when not.

FYI: There's another project, `ods.stac` that does something similar. Check out _their_ [documentation](https://odc-stac.readthedocs.io/en/latest/).

TBD which one wins :)

In [ ]:
import stackstac

# bounds: (min_x, min_y, max_x, max_y)
# https://linestrings.com/bbox/#-157,71.2,-156.5,71.35
# epsg = 32604 -- native
# epsg = 4326 wgs84
# epsg = 3995
# 
east, west, north, south = -156.7, -156.8, 71.3, 71.25
stack = stackstac.stack(
    item_collection[:10],
    assets = ['nir'],
    bounds_latlon = [west, south, east, north],
    epsg = 4326,
    #resolution = 0.001
)
stack

In [ ]:
stack.isel(time=slice(0, 6)).squeeze().plot.imshow(col="time", col_wrap=3, robust=True)

# California Forest Observatory


This one is an example of a _static_ STAC catalog. No API server here! Just a plain ol' JSON file in cloud storage, with links to other JSON files. We lose a lot of the rich interactivity we had with STAC APIs, but we can still manually navigate the catalog to learn about the items within it, and extract URLs to associated data assets.

Open this in your web browser:
https://storage.googleapis.com/cfo-public/catalog.json

In [ ]:
uri = 'https://storage.googleapis.com/cfo-public/catalog.json'
catalog = Client.open(uri)
catalog

In [ ]:
collections = catalog.get_all_collections()
for collection in collections:
    print(f'[{collection.id}]: {collection.title}')

In [41]:
wildfire_collection = catalog.get_collection('wildfire').get_items()

In [ ]:
wildfire_items = [item for item in wildfire_collection]
wildfire_items

In [ ]:
list(wildfire_items)[0]

In [ ]:
wildfire_assets = catalog \
    .get_collection('wildfire') \
    .get_item('California-Wildfire-BurnProbability-2020-Summer-00010m') \
    .assets
wildfire_assets

In [ ]:
burn_probability_asset = wildfire_assets['BurnProbability']
burn_probability_asset

In [ ]:
href = catalog \
    .get_collection('wildfire') \
    .get_item('California-Wildfire-BurnProbability-2020-Summer-00010m') \
    .assets['BurnProbability'] \
    .href
href

In [ ]:
ds = xr.open_dataset(href, engine='rasterio', chunks={}).band_data
ds